<html>
    <div dir="rtl">
    در این نسخه از pos tag shuffler می‌خواهیم از یک مدل زبانی n-gram استفاده کنیم برای یافتن بهترین ترکیب pos tag ها
    بدین شکل که تمامی جایگشت‌های ممکن از pos tag ها را می‌سازیم و با استفاده از این مدل زبانی به آن‌ها امتیاز می‌دهیم سپس بهترین ها را به مدل زبانی اصلی می‌دهیم.
    </div>
</html>

In [10]:
from hazm import POSTagger, Normalizer
from enum import Enum, auto
from hazm import SentenceTokenizer, WordTokenizer
from tqdm import tqdm
import itertools
tagger = POSTagger(model="resources/pos_tagger.model")

In [11]:
n = 3

In [12]:
def make_n_gram_model(n, resource='resources/Datasets/MirasText_sample.txt'):
    with open(resource, 'r') as f:
        text = f.read()
    sentences = SentenceTokenizer().tokenize(text)
    tri_grams = []
    for sentece in tqdm(sentences):
        words = WordTokenizer().tokenize(sentece)
        tagged_words = tagger.tag(words)

        # make 3-gram from tags
        tags = ['START'] + [tag for word, tag in tagged_words] + ['END']
        ng = ngrams(tags, n)
        tri_grams.extend(ng)
    return tri_grams

In [13]:
def get_probability_dic(n):
    probability_dic = {}
    ngram_model = make_n_gram_model(n)
    for tup in ngram_model:
        probability_dic[tup] = probability_dic.get(tup, 0.01) + 1
        if tup[0] == 'START':
            probability_dic[tup] += 100
    return probability_dic
    

In [14]:
import pickle

dataset_action = "load"

if dataset_action == "build":
    probability_dic = get_probability_dic(n)
    dataset_action = "save"

if dataset_action == "save" and probability_dic:
    with open("resources/Datasets/ngram_shuffler.pickle", "wb") as f:
        pickle.dump(probability_dic, f)
        print("Saved dataset to pickle file!")
elif dataset_action == "load":
    with open("resources/Datasets/ngram_shuffler.pickle", "rb") as f:
        probability_dic = pickle.load(f)
        print("Loaded dataset from pickle file!")

Loaded dataset from pickle file!


In [15]:
  def calculate_probability_of_postags(n, postags: list):
        probability = 1
        tags = ['START'] + postags + ['END']
        for i in range(len(tags) - (n-1)):
            probability *= probability_dic.get(tuple(tags[i:i+n]), 1)
        return probability

In [16]:
def get_possible_permutations(n, sentence: str):
    words = WordTokenizer().tokenize(sentence)
    tagged_words = tagger.tag(words)
    print(tagged_words)
    permutations = itertools.permutations(tagged_words)
    all_possibilities = [(permutation, calculate_probability_of_postags(n, [tag for word, tag in permutation])) for permutation in permutations]
    return sorted(all_possibilities, key=lambda x: x[1], reverse=True)[:10]

In [17]:
text = "علی با من حرف زد."
tagged_text = tagger.tag(WordTokenizer().tokenize(text))
print(tagged_text)
print(calculate_probability_of_postags(n, [tag for word, tag in tagged_text]))


[('علی', 'NOUN'), ('با', 'ADP'), ('من', 'PRON'), ('حرف', 'NOUN'), ('زد', 'VERB'), ('.', 'PUNCT')]
3.2016573093341353e+22


In [18]:
permutations = get_possible_permutations(n, "من و علی رفتیم به مدرسه.")
for permutation in permutations:
    print(' '.join([word for word, tag in permutation[0]]))

[('من', 'PRON'), ('و', 'CCONJ'), ('علی', 'NOUN'), ('رفتیم', 'VERB'), ('به', 'ADP'), ('مدرسه', 'NOUN'), ('.', 'PUNCT')]
من به علی و مدرسه رفتیم .
من به مدرسه و علی رفتیم .
من و علی به مدرسه رفتیم .
من و مدرسه به علی رفتیم .
و من به علی مدرسه رفتیم .
و من به مدرسه علی رفتیم .
و علی به من مدرسه رفتیم .
و مدرسه به من علی رفتیم .
من و به علی مدرسه رفتیم .
من و به مدرسه علی رفتیم .
